In [1]:
''' 
Minflux explorer v0.1
2D minflux data only!
Running script/opening page automatically prompts selection of a single *.npy file.
Todo: figure out how to do this semi-ineractively on the dashboard

Contact christopher.tynan@stfc.ac.uk for help.
'''

from tkinter import Tk, filedialog
from IPython.display import clear_output
import numpy as np
from tqdm import tqdm
import pandas as pd
import hvplot.pandas
import holoviews as hv
import holoviews.operation.datashader as hd
import datashader as ds
import colorcet as cc
import panel as pn
import nbconvert


def minflux_npy_to_df(npyfile):
    tid = npyfile['tid'] #consecutive minflux measurements are attributed to the same emitter
    vld = npyfile['vld'] #not all minflux measurement succeed and have a valid localisation 
    tim = npyfile['tim']
    loc = npyfile['itr']['loc'][:, 4, :] # get localisations from the 4th and final search iteration
    loc_nm = loc*1e09 # convert positions to nm
    dcr = npyfile['itr']['dcr'][:, 4] # which channel for 2c minflux
    efo = npyfile['itr']['efo'][:, 4]
    cfr = npyfile['itr']['cfr'][:, 4]
    
    df = pd.DataFrame({'tid': tid, 'vld': vld, 'tim': tim, 'dcr': dcr, 'x': loc_nm[:, 0], 'y': loc_nm[:, 1], 'z': loc_nm[:, 2], 'efo': efo, 'cfr': cfr})
    vld_df = df[df['vld']==True]
    return vld_df


def extract_trace_statistics(df, minlen):
    traces =  df.groupby(['tid'], as_index=False).filter(lambda x: len(x) > minlen)
    means = traces.groupby(['tid'], as_index=False).mean()
    stds = traces.groupby(['tid'], as_index=False).std()
    return traces, means, stds


# First select *.npy file to explore 
clear_output()
root = Tk()
root.withdraw() # Hide the main window.
root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
file = filedialog.askopenfilename(multiple=False) # List of selected files will be set button's file attribute.

print("Selected file: ", file)

mf_df = minflux_npy_to_df(np.load(file))
max_tim = mf_df['tim'].max()


# get min/max spatial coordinates and calculate range for setting image sizes
xmin = mf_df['x'].min()
xmax = mf_df['x'].max()
ymin = mf_df['y'].min()
ymax = mf_df['y'].max()

xrange = xmax - xmin
yrange = ymax - ymin


# filter localisations by efo, cfr and time
filtered_df = mf_df[(mf_df.efo < 70000) & (mf_df.cfr < 0.35)]


# use Datashader to plot rasterised image of localisations (equivalent to Abberior Imspektor live view)
bin_size = 4 # binsize in nm
w = round(xrange/bin_size)
h = round(yrange/bin_size)

agg = ds.Canvas(plot_width=w, plot_height=h).points(filtered_df, 'x', 'y')
hv.output(backend="bokeh")
mf_recon = hd.shade(hv.Image(agg), cmap=cc.bmy, clim=(0, 10)).redim.nodata(value=None).opts(bgcolor='black', frame_width=600, frame_height=600, aspect='equal')


# use dataframe.groupby(['tid']) to get stats for each trace, then plot mean x,y precision with ellipse for precision (1 standard deviation)
minlen = 5
traces, means, stds = extract_trace_statistics(filtered_df, minlen)

locs = traces.hvplot.points(x='x', y='y', c='tim', cmap=cc.rainbow).opts(marker='x', size=5, frame_width=600, frame_height=600, aspect='equal')

centres = means.hvplot.points(x='x', y='y', c='tim', cmap=cc.rainbow).opts(marker='+', size=10, frame_width=600, frame_height=600, aspect='equal')

ellipses = hv.Polygons([hv.Ellipse(means['x'][i], means['y'][i], (stds['x'][i], stds['y'][i])) for i in range(len(means['x']))])
ellipses_plot = ellipses.opts(line_width=1, frame_width=600, frame_height=600, aspect='equal', color=None)

# lump it together and let holoviews do it's magic for linking and brushing
linked_plot = mf_recon + (locs * centres * ellipses_plot)

# have it as a little dashboard to push to a new browser tab
header = '##MINFLUX explorer v0.1: ' + file
mf_explorer_dashboard = pn.Column(header, linked_plot)
mf_explorer_dashboard.show()

Selected file:  C:/Users/ct83/Documents/Data analysis/Minflux/EGFR/Selene_20220330/220331-121506_minflux.npy
Launching server at http://localhost:63430
